In [1]:
%load_ext autoreload

In [2]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role
import rrt, prm, utils, rrtiris
from sandbox.t_space_utils import (convert_t_to_q, 
                                   convert_q_to_t)

from utils import meshcat_line
from iris_t_space import set_up_iris_t_space, RegionCertifier, EvaluatePlanePair, uniform_shrink_iris_region_list, uniform_shrink_iris_region

In [3]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

# Build plant and simulation

In [4]:
# Build plant and simulation#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]
t_low = convert_q_to_t(np.array(q_low).reshape(1,-1)).squeeze()
t_high = convert_q_to_t(np.array(q_high).reshape(1,-1)).squeeze()

#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 50

In [10]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
def set_joint_angles(vals):
    joints[0].set_angle(plant_context, vals[0])
    joints[1].set_angle(plant_context, vals[1])
    joints[2].set_angle(plant_context, vals[2])
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))

col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def eval_cons_rational(t0, t1, t2, c, tol):
    q = convert_t_to_q(np.array([t0, t1, t2]).reshape(1,-1)).squeeze() 
    return col_func_handle(*q)
   
col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)

def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    t = convert_q_to_t(np.array(q).reshape(1,-1)).squeeze()
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
#     plane_callback(convert_q_to_t(q))
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [13]:
def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    t = convert_q_to_t(np.array(q).reshape(1,-1)).squeeze()
#     plane_callback(t)
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


In [14]:
#marching cubes
N = 50
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), N, N, N, col_func_handle_rational, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
q = q0.copy()
showres(q)

In [15]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q2', max=2.0, min=-2.0)

In [16]:
#SPP + IRIS / RRT / PRM  CONFIG + viz
start_q = np.array([0.0, -1.6, 1.5])
target_q = np.array([0.7,-0.9,1.5])
start = convert_q_to_t(start_q.reshape(1,-1))
target = convert_q_to_t(target_q.reshape(1,-1))

#
#plot start and target
mat = meshcat.geometry.MeshLambertMaterial(color=0xFFDD36)
mat.reflectivity = 1.0
vis2['start'].set_object(
                meshcat.geometry.Sphere(0.03), mat)
vis2['start'].set_transform(
                meshcat.transformations.translation_matrix(start.reshape(-1,)))

mat = meshcat.geometry.MeshLambertMaterial(color=0x06D300)
mat.reflectivity = 1.0
vis2['target'].set_object(
                meshcat.geometry.Sphere(0.03), mat)
vis2['target'].set_transform(
                meshcat.transformations.translation_matrix(target.reshape(-1,)))

def draw_traj_tspace(traj, maxit, name):
    #evals end twice fix later
    for it in range(maxit):
        pt = traj.value(it*traj.end_time()/maxit)
        pt_nxt = traj.value((it+1)*traj.end_time()/maxit)
        
        pt_q = convert_t_to_q(pt.reshape(1,-1)).squeeze()

        mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
        mat.reflectivity = 1.0
        vis2[name]['traj']['points' + str(it)].set_object( meshcat_line(pt.squeeze(), pt_nxt.squeeze(),width = 0.03), mat)
        
        set_joint_angles(pt_q.reshape(-1,))
        tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
        R_l2 = tf_l2.rotation()
        tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()

        tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
        R_la = tf_la.rotation()
        tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()


        mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
        mat.reflectivity = 1.0
        vis[name]['traj']['link2']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis[name]['traj']['link2']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_l2))
        mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
        mat.reflectivity = 1.0
        vis[name]['traj']['linka']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis[name]['traj']['linka']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_la))

# Construct rational forward kin things

In [ ]:
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere, VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable,
    BsplineTrajectoryThroughUnionOfHPolyhedra,
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff, InverseKinematics,
    RationalForwardKinematics, FindBodyInTheMiddleOfChain
)
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import pydrake
import meshcat

from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable, Expression, RotationMatrix
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere, VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff, InverseKinematics
)

import pydrake.symbolic as sym
import symbolic_parsing_helpers as symHelpers
from pydrake.all import RationalForwardKinematics, FindBodyInTheMiddleOfChain
from pydrake.all import GenerateMonomialBasisOrderAllUpToOneExceptOneUpToTwo, GenerateMonomialBasisWithOrderUpToOne

#copy construct of region certifier
def MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)
def MakeFromVPolytopeOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return VPolytope(query, geom, expressed_in)


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
q_star = np.zeros(forward_kin.t().shape[0])
inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()

# geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
pair_set = set()
for p in pairs:
    pair_set.add(p[0])
    pair_set.add(p[1])
geom_ids = inspector.GetGeometryIds(GeometrySet(list(pair_set)))

HPolyhedronSets = {geom: MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for
                   geom in geom_ids}
VPolyhedronSets = {geom: MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom
                   in geom_ids}

body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in
                           geom_ids}
forward_kin = RationalForwardKinematics(plant)
convSolver = MosekSolver()
t_kin = forward_kin.t()

link_poses_by_body_index_multilinear_pose = forward_kin.CalcLinkPosesAsMultilinearPolynomials(np.zeros_like(t_kin),
                                                                                              plant.world_body().index())
link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
X_WA_multilinear_list = [(r.rotation().copy(), r.translation().copy()) for r in
                         link_poses_by_body_index_multilinear_pose]
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret
X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)


## CONSTRUCT T SPACE IRIS ##


In [19]:
iris_rational_space, query, forward_kin = set_up_iris_t_space(plant, scene_graph, context, settings = None)

def do_iris(t_seed, verbose = False):
        start_time = time.time()
        hpoly = iris_rational_space(query, t_seed, require_containment_points=[t_seed], iteration_limit=100)
        ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
        if verbose:
            print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
              ellipse.center(), flush=True)
        return hpoly, ellipse

In [23]:
USE_HAND_CRAFTED = True

seed_points_q = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]])[::-1, :]    # passing

seed_points = convert_q_to_t(seed_points_q)

def rejection_sampling_iris(its):
    regions = []
    ellipses = []
    samples = []
    seed_points = [start.squeeze(), target.squeeze()]
    reg, ell = do_iris(seed_points[0])
    regions.append(reg)
    ellipses.append(ell)
    reg, ell = do_iris(seed_points[1])
    regions.append(reg)
    ellipses.append(ell)
    
    for _ in range(its):
        #rejection sampling to get initial feasible point 
        found = False
        while not found:
            t = np.random.rand(3)
            t_samp = (1-t)*t_low + t*t_high
            found = (col_func_handle_rational(*t_samp)==0.0)
        print("point found: ", t_samp)
        seed_points.append(t_samp)
        reg, ell = do_iris(t_samp)
    
        regions.append(reg)
        ellipses.append(ell)
    return regions, ellipses, seed_points

def hand_crafted_seedpoint_iris(seed_points):
    regions = []
    ellipses = []
    for i in range(seed_points.shape[0]):
        reg, ell = do_iris(seed_points[i, :])
        regions.append(reg)
        ellipses.append(ell)
    print("SUCCESS")
    return regions, ellipses, seed_points 

if USE_HAND_CRAFTED:
    regions, ellipses, region_seeds = hand_crafted_seedpoint_iris(seed_points)
else:
    regions, ellipses, region_seeds = rejection_sampling_iris(15)

region_seeds = np.array(region_seeds)
for i in range(region_seeds.shape[0]):
    vis2['iris']['seedpoints']["seedpoint"+str(i)].set_object(
                meshcat.geometry.Sphere(0.05), meshcat.geometry.MeshLambertMaterial(color=0x0FB900))
    vis2['iris']['seedpoints']["seedpoint"+str(i)].set_transform(
                meshcat.transformations.translation_matrix(region_seeds[i,:]))

0
terminating because a required containment point would have not been contained
0
1
0
1
0
1
0
1
SUCCESS


## Plotting Resulting Regions ##


In [25]:
idx = 0
vis2['iris'].delete()
for region in regions:
    c1 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    c2 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    c3 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((c1, c2,c3)), wireframe=True)
    mat.opacity = 0.3
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis2['iris']['regions'],
                       name = str(idx),
                       mat = mat)
    
    C = ellipses[idx].A()#[:, (0,2,1)]
    d = ellipses[idx].center()#[[0,2,1]]
    radii, R = np.linalg.eig(C.T@C)
    R[:,0] = R[:,0]*np.linalg.det(R)
    Rot = RotationMatrix(R)
 
    transf = RigidTransform(Rot, d)
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((c1, c2,c3)), wireframe=True)
    mat.opacity = 0.15
    vis2['iris']['ellipses'][str(idx)].set_object(
            meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
            mat)

    vis2['iris']['ellipses'][str(idx)].set_transform(transf.GetAsMatrix4())
    
    idx+=1

In [29]:
def construct_first_order_separating_hyperplane(prog, t, order = 2, plane_name = ''):
    if plane_name != '':
        plane_name = '_'+plane_name
    t_basis = sym.MonomialBasis(t, order)
    t_basis = np.array([sym.Polynomial(v) for v in t_basis])
    a_A_coeffs = prog.NewContinuousVariables(3, t_basis.shape[0], 'a_A'+plane_name)
    a_poly = a_A_coeffs@t_basis
    
    b_A_coeffs = prog.NewContinuousVariables(1, t_basis.shape[0], 'b_A'+plane_name)
    b_poly = b_A_coeffs@t_basis
    
    for i, p in enumerate(a_poly):
        a_poly[i].SetIndeterminates(sym.Variables(t))
    for i, p in enumerate(b_poly):
        b_poly[i].SetIndeterminates(sym.Variables(t))
    dec_vars = [*a_A_coeffs.flatten().tolist(), *b_A_coeffs.flatten().tolist()]
    ntmp = a_A_coeffs.flatten().shape[0]
    Qtmp, btmp = np.eye(ntmp), np.zeros(ntmp)
    prog.AddQuadraticCost(Qtmp, btmp, a_A_coeffs.flatten())
    return prog, a_poly, b_poly.item(), dec_vars


def putinarPsatConstraint(prog, p, t,  
                         poly_to_cert, lagrange_mult_degree = 2, var_epsilon = None):
    A = poly_to_cert.A()
    b = poly_to_cert.b()
    n = b.shape[0]
    if var_epsilon is None:
        var_epsilon = np.zeros(n)
#     lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
    lagrange_poly, Q = prog.NewSosPolynomial(GenerateMonomialBasisOrderAllUpToOneExceptOneUpToTwo(
        sym.Variables(t)))
    lagrange_poly.SetIndeterminates(sym.Variables(t))
    prog.AddSosConstraint(lagrange_poly)
    constraint_poly = lagrange_poly
    for i in range(n):
#         lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        lagrange_poly, Q = prog.NewSosPolynomial(
            GenerateMonomialBasisOrderAllUpToOneExceptOneUpToTwo(sym.Variables(t)))
        lagrange_poly.SetIndeterminates(sym.Variables(t))
        prog.AddSosConstraint(lagrange_poly)
        constraint_poly += lagrange_poly*sym.Polynomial(b[i]-var_epsilon[i]-A[i,:]@t)
    constraint_poly.SetIndeterminates(sym.Variables(t))
    tol = 1e-3
    prog.AddEqualityConstraintBetweenPolynomials(constraint_poly, p-tol)
    return prog, (constraint_poly, p-tol)


def makeBodyHyperplaneSidePolynomials(prog, a_plane_poly, b_plane_poly,
                          VPoly, R_WA, p_WA, t, poly_to_cert, leq_or_geq,
                          lagrange_mult_degree = 2, var_epsilon = None, base_point_as_multilinear_poly = None):
    num_verts = VPoly.vertices().shape[1]

    vertex_pos = R_WA@(VPoly.vertices())+ np.repeat(p_WA[:, np.newaxis],num_verts,1)
    zero_poly = sym.Polynomial(0)
    if base_point_as_multilinear_poly is None:
        base_point_as_multilinear_poly = np.array([zero_poly for i in range(3)])
    
    
    dens = np.ndarray(shape = vertex_pos.shape,dtype = object)
    nums = np.ndarray(shape = vertex_pos.shape,dtype = object)
    for i, row in enumerate(vertex_pos):
        
        for j, v in enumerate(row):
            vertex_pos[i,j] = forward_kin.ConvertMultilinearPolynomialToRationalFunction(v)
            dens[i,j] = vertex_pos[i,j].denominator()
            nums[i,j] = vertex_pos[i,j].numerator()

    unique_dens_c = [[sym.Polynomial(1)] for _ in range(dens.shape[1])]
    col_den = [sym.Polynomial(1) for _ in range(dens.shape[1])]
    for c in range(dens.shape[1]):
        for r in range(dens.shape[0]):
            is_unique = True
            for d in unique_dens_c[c]:
                is_unique = False if dens[r,c].EqualTo(d) else True
                if not is_unique:
                    break
            if is_unique:
                unique_dens_c[c].append(dens[r,c] )
    
        for d in unique_dens_c[c]:
            col_den[c] *= d

    for c in range(nums.shape[1]):
        for r in range(nums.shape[0]):
            for d in unique_dens_c[c]:
                if not dens[r,c].EqualTo(d):
                    nums[r,c] *= d

    plane_polys = np.array([None for _ in range(vertex_pos.shape[1])])
    prog_eps = 1e-12
    for i in range(vertex_pos.shape[1]):
        if leq_or_geq == 'leq':
            #a^Tx + b <= -1 -> -(a^Tx+b+1)>=0
            plane_polys[i] = -a_plane_poly.dot(nums[:,i])-(b_plane_poly+1)*(col_den[i])

        elif leq_or_geq == 'geq':
            #a^Tx+b >= 1 -> a^Tx+b -1 >= 0
            plane_polys[i] = a_plane_poly.dot(nums[:,i])+(b_plane_poly-1)*(col_den[i])
        else:
            raise ValueError("leq_or_geq arg must be leq or geq not {}".format(leq_or_geq))  
        plane_polys[i].SetIndeterminates(sym.Variables(t))

    return plane_polys


def add_pair_constraint(geomA, geomB, prog, poly_to_cert, 
                        lagrange_mult_degree = 2, var_epsilon = None):
    VPolyA, VPolyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]
    prog, a_plane_poly, b_plane_poly, dec_vars = construct_first_order_separating_hyperplane(prog, t_kin)
        
    R_WA, p_WA = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomA])]
    R_WB, p_WB  = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomB])]
    
    base_point_poly = None#R_WA@VPolyA.vertices().mean(axis = 1)-p_WA
#     base_point = np.array([forward_kin.ConvertMultilinearPolynomialToRationalFunction(p) for p in base_point_poly])
    
    
    plane_polys_A = makeBodyHyperplaneSidePolynomials(prog, a_plane_poly, b_plane_poly, 
                                                     VPolyA, R_WA, p_WA , t_kin, 
                                                     poly_to_cert, 'leq', lagrange_mult_degree,
                                                     var_epsilon, base_point_poly)
    plane_polys_B = makeBodyHyperplaneSidePolynomials(prog, a_plane_poly, b_plane_poly,
                                                     VPolyB, R_WB, p_WB , t_kin, 
                                                     poly_to_cert, 'geq', lagrange_mult_degree,
                                                     var_epsilon, base_point_poly)
    plane_polys = np.array(plane_polys_A.tolist()+ plane_polys_B.tolist()).squeeze()
    
    
    s_prod_pairs = []
    
    for p in plane_polys:
        prog, (constraint_poly, p_with_tol) = putinarPsatConstraint(prog, p, t_kin,  
                         poly_to_cert, lagrange_mult_degree = lagrange_mult_degree, var_epsilon = var_epsilon)
        s_prod_pairs.append((constraint_poly, p_with_tol))
    return prog, plane_polys, (a_plane_poly, b_plane_poly), dec_vars, s_prod_pairs


def construct_region_safety_problem(poly_to_cert, lagrange_mult_degree = 2, 
                                    var_epsilon = None, check_archimedean = True,
                                    plane_name = ""):
    
    if check_archimedean:
        is_Archimedean,_ = certify_archimedean(poly_to_cert, t_kin)
        if not is_Archimedean:
            raise ValueError("region is not archimedean")
    prog = MathematicalProgram()
    prog.AddIndeterminates(t_kin)
    plane_pairs = {}
    plane_polys_list = []
    s_prod_pairs = {}
    dec_vars = []
    for i, (geomA, geomB) in enumerate(pairs):
        print("pair {}/{}".format(i+1, len(pairs)))
        prog, plane_polys, (a_poly, b_poly), dec_vars0, s_prod_pairs0 = add_pair_constraint(geomA, geomB,
                                                       prog, poly_to_cert,
                                                       lagrange_mult_degree = lagrange_mult_degree,
                                                       var_epsilon = var_epsilon
                                                      )
        plane_polys_list.append(plane_polys)
        plane_pairs[(geomA, geomB)] = (a_poly, b_poly)
        s_prod_pairs[(geomA, geomB)] = s_prod_pairs0
        dec_vars += dec_vars0
#         if i >= 0:
#             break
    return prog, plane_pairs, plane_polys_list, np.array(dec_vars), s_prod_pairs

def construct_region_safety_problems_pairwise(poly_to_cert, lagrange_mult_degree = 2, 
                                    var_epsilon = None, check_archimedean = True,
                                    plane_name = ""):
    if check_archimedean:
        is_Archimedean,_ = certify_archimedean(poly_to_cert, t_kin)
        if not is_Archimedean:
            raise ValueError("region is not archimedean")
    
    progs = {}
    for i, (geomA, geomB) in enumerate(pairs):
        prog = MathematicalProgram()
        prog.AddIndeterminates(t_kin)
        plane_pairs = {}
        plane_polys_list = []
        s_prod_pairs = {}
        dec_vars = []
        print("pair {}/{}".format(i+1, len(pairs)))
        prog, plane_polys, (a_poly, b_poly), dec_vars0, s_prod_pairs0 = add_pair_constraint(geomA, geomB,
                                                       prog, poly_to_cert,
                                                       lagrange_mult_degree = lagrange_mult_degree,
                                                       var_epsilon = var_epsilon
                                                      )
        plane_polys_list.append(plane_polys)
        plane_pairs[(geomA, geomB)] = (a_poly, b_poly)
        s_prod_pairs[(geomA, geomB)] = s_prod_pairs0
        dec_vars += dec_vars0
        progs[(geomA, geomB)] = (prog, (plane_pairs, plane_polys_list, np.array(dec_vars), s_prod_pairs))
#         if i >= 0:
#             break
    return progs



def extract_planes(plane_pairs, result):
    resulting_plane_pairs = {}

    for k, (a, b) in plane_pairs.items():
        a_list = []
        for ai in a:
            a_list.append(result.GetSolution(ai))
        resulting_plane_pairs[k] = (np.array(a_list), result.GetSolution(b))
    return resulting_plane_pairs


def extract_s_prod_pairs(s_prod_pairs, result):
    evaled_pairs = {}
    for k, poly_list in s_prod_pairs.items():
        cur_list = []
        for (const, vert) in poly_list:
            constraint_poly_evaled = result.GetSolution(const)
            vert_poly_evaled = result.GetSolution(vert)
            cur_list.append((constraint_poly_evaled, vert_poly_evaled))
        evaled_pairs[k] = cur_list
    return evaled_pairs


def certify_region(poly_to_cert, lagrange_mult_degree = 4, var_epsilon = None, check_archimedean = True, numeric_tol = 1e-10):
    prog, plane_pairs, plane_polys_list, dec_vars, s_prod_pairs = construct_region_safety_problem(poly_to_cert,
                                                                                                      lagrange_mult_degree=lagrange_mult_degree,
                                                                                                      var_epsilon=var_epsilon,
                                                                                                      check_archimedean=check_archimedean)
    result = convSolver.Solve(prog)

    if result.is_success():
        plane_polys = extract_planes(plane_pairs, result)
        s_prod_pairs = extract_s_prod_pairs(s_prod_pairs, result)
    else:
        plane_polys = None
        s_prod_pairs = None
    return result, plane_polys, s_prod_pairs

def certify_regions_pairwise(poly_to_cert, lagrange_mult_degree = 4, var_epsilon = None, check_archimedean = True, numeric_tol = 1e-10):
    progs = construct_region_safety_problems_pairwise(poly_to_cert,
                                                    lagrange_mult_degree=lagrange_mult_degree,
                                                    var_epsilon=var_epsilon,
                                                    check_archimedean=check_archimedean)
    
    result_dict = dict.fromkeys(progs.keys())
    all_cert = True
    for key, (prog, (plane_pairs, plane_polys_list, dec_vars, s_prod_pairs)) in progs.items():
                                                                                                        
        result = convSolver.Solve(prog)

        if result.is_success():
            plane_polys = extract_planes(plane_pairs, result)
            s_prod_pairs = extract_s_prod_pairs(s_prod_pairs, result)
        else:
            plane_polys = None
            s_prod_pairs = None
            all_cert = False
            break
        result_dict[key] = (result, plane_polys, s_prod_pairs)
    return result_dict, all_cert

def certify_archimedean(poly_to_cert, t, lagrange_mult_degree = 2):
    #need to fix
    convSolver = MosekSolver()
    prog = MathematicalProgram()
    prog.AddIndeterminates(t)
        
    t_bar = sym.Polynomial(t@t)

    poly = t_bar
    A = poly_to_cert.A()
    b = poly_to_cert.b()
    n = b.shape[0]
    for i in range(n):
        lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        lagrange_poly.SetIndeterminates(sym.Variables(t))
        prog.AddSosConstraint(lagrange_poly)
        poly += lagrange_poly*sym.Polynomial(b[i] - A[i,:]@t)
    poly.SetIndeterminates(sym.Variables(t))
    prog.AddSosConstraint(poly)
    result = convSolver.Solve(prog)
    return result.is_success(), prog


In [30]:
regions_shrunk = uniform_shrink_iris_region_list(regions, var_epsilon=5e-2)
idx = 0

for region in regions_shrunk:
    c1 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    c2 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    c3 = int(np.clip(255*np.random.rand(), a_min = 0, a_max = 255))
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((c1, c2,c3)), wireframe=True)
    mat.opacity = 0.6
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis2['iris']['regions'],
                       name = str(idx),
                       mat = mat)
    
    idx+=1


In [31]:

certified_regions_polys = {}
uncertified_regions = []
for r in regions_shrunk:
    result_dict, all_cert = certify_regions_pairwise(r)

    print(all_cert)
    if all_cert is True:
        certified_regions_polys[r] = plane_polys
    else:
        uncertified_regions.append(r)
print("done")

pair 1/14
pair 2/14
pair 3/14
pair 4/14
pair 5/14
pair 6/14
pair 7/14
pair 8/14
pair 9/14
pair 10/14
pair 11/14
pair 12/14
pair 13/14
pair 14/14
True
pair 1/14
pair 2/14
pair 3/14
pair 4/14
pair 5/14
pair 6/14
pair 7/14
pair 8/14
pair 9/14
pair 10/14
pair 11/14
pair 12/14
pair 13/14
pair 14/14
False
pair 1/14
pair 2/14
pair 3/14
pair 4/14
pair 5/14
pair 6/14
pair 7/14
pair 8/14
pair 9/14
pair 10/14
pair 11/14
pair 12/14
pair 13/14
pair 14/14
True
pair 1/14
pair 2/14
pair 3/14
pair 4/14
pair 5/14
pair 6/14
pair 7/14
pair 8/14
pair 9/14
pair 10/14
pair 11/14
pair 12/14
pair 13/14
pair 14/14
False
pair 1/14
pair 2/14
pair 3/14
pair 4/14
pair 5/14
pair 6/14
pair 7/14
pair 8/14
pair 9/14
pair 10/14
pair 11/14
pair 12/14
pair 13/14
pair 14/14
True
True
done


In [263]:
for r in regions_shrunk:
    print(r.PointInSet(target.squeeze()))

False
False
False
True
False


In [293]:
certified_regions_polys.keys()

dict_keys([<pydrake.geometry.optimization.HPolyhedron object at 0x7f994c9e03f0>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f994c727f30>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f994c6cbf30>])

In [247]:
planes = list(certified_regions_polys.values())[0]

In [248]:
print(planes)
print()
print(pairs)

{(<GeometryId value=132>, <GeometryId value=139>): (array([<Polynomial "3.3159047578376086e-09*1 + 1.1753447816423315e-09*t[2] + -5.9386442495788174e-10*t[2]^2 + 1.5416260439742824e-09*t[1] + 1.6740010750774863e-09*t[1] * t[2] + -3.6107213105428993e-10*t[1]^2 + 2.348377679628282e-10*t[0] + -3.1500176023833279e-10*t[0] * t[2] + 2.408186171793954e-09*t[0] * t[1] + -1.4904537542051707e-09*t[0]^2">,
       <Polynomial "-12.111934238001417*1 + 4.9290243970543335*t[2] + -2.0393288317041605*t[2]^2 + 5.6987082085039802*t[1] + -2.1549542807400486*t[1] * t[2] + -2.8836591711213471*t[1]^2 + -3.8149640937836558*t[0] + 1.472334263052961*t[0] * t[2] + 2.0537281495207429*t[0] * t[1] + -1.3691411322067879*t[0]^2">,
       <Polynomial "0.11529048107181762*1 + -2.0247131710572255*t[2] + 1.6157726508376189*t[2]^2 + -6.7446683089352213*t[1] + 4.3926514567577231*t[1] * t[2] + 8.9773584630439363*t[1]^2 + -0.42178721883641662*t[0] + -0.54945247698501321*t[0] * t[2] + -1.7685374187071188*t[0] * t[1] + -0.3741

In [28]:
t_space_vertex_position_by_geom_id ={}
for geom in geom_ids:
    VPoly = VPolyhedronSets[geom]
    num_verts = VPoly.vertices().shape[1]
    X_WA = X_WA_list[int(body_indexes_by_geom_id[geom])]
    R_WA = X_WA.rotation().matrix()
    p_WA = X_WA.translation()
    vert_pos = R_WA@(VPoly.vertices())+ np.repeat(p_WA[:, np.newaxis],num_verts,1)
    t_space_vertex_position_by_geom_id[geom] = vert_pos

In [327]:
import scipy 
import utils

x = np.linspace(-1,1, 3)
y = np.linspace(-1,1, 3)
verts = []

for idxx in range(len(x)):
    for idxy in range(len(y)):
        verts.append(np.array([x[idxx], y[idxy]]))
       
tri = scipy.spatial.Delaunay(verts)
plane_triangles = tri.simplices
plane_verts = tri.points[:,:]
plane_verts = np.concatenate((plane_verts, 0*plane_verts[:,0].reshape(-1,1)), axis = 1)

def transform(a,b, p1, p2, plane_verts, plane_triangles):
    alpha = (-b-a.T@p1)/(a.T@(p2-p1))
    offset = alpha*(p2-p1) + p1
    z = np.array([0, 0, 1])
    crossprod = np.cross(utils.normalize(a), z)
    if np.linalg.norm(crossprod) <=1e-4:
        R = np.eye(3)
    else:
        ang = np.arcsin(np.linalg.norm(crossprod))
        axis = utils.normalize(crossprod)
        R = utils.get_rotation_matrix(axis, -ang)
        
    verts_tf = (R@plane_verts.T).T + offset
    return verts_tf

def transform_at_t(cur_t, a_poly, b_poly, p1_rat, p2_rat):
    eval_dict = dict(zip(sym.Variables(t_kin), cur_t))
    a,b = EvaluatePlanePair((a_poly, b_poly), eval_dict)
#     print(f"{a}, {b}")
    p1 = np.array([p.Evaluate(eval_dict) for p in p1_rat])
    p2 = np.array([p.Evaluate(eval_dict) for p in p2_rat])
    return transform(a,b, p1, p2, plane_verts, plane_triangles), p1, p2

def transform_plane_geom_id(geomA, geomB, planes_dict, cur_t):
    vA = t_space_vertex_position_by_geom_id[geomA][:,0]
    vB = t_space_vertex_position_by_geom_id[geomB][:,0]
    a_poly, b_poly = planes_dict[(geomA, geomB)]
    return transform_at_t(cur_t, a_poly, b_poly, vA, vB)

def plot_plane_geom_id(geomA, geomB, planes_dict, cur_t, color = (0,0,0)):
    verts_tf, p1, p2 = transform_plane_geom_id(geomA, geomB, planes_dict, cur_t)    
    
    mat = meshcat.geometry.MeshLambertMaterial(color=utils.rgb_to_hex(color), wireframe=False)
    mat.opacity = 0.5
    vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"].set_object(
                meshcat.geometry.TriangularMeshGeometry(verts_tf, plane_triangles),
                mat)
    
    mat.opacity = 1.0
    utils.plot_point(loc = p1, radius = 0.05, mat = mat, vis = vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"], marker_id= 'p1')
    mat = meshcat.geometry.MeshLambertMaterial(color=utils.rgb_to_hex(color), wireframe=False)
    mat.opacity = 1.0
    utils.plot_point(loc = p2, radius = 0.05, mat = mat, vis = vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"], marker_id= 'p2')
    

In [304]:
import colorsys
import itertools
from fractions import Fraction
def infinite_hues():
    yield Fraction(0)
    for k in itertools.count():
        i = 2**k # zenos_dichotomy
        for j in range(1,i,2):
            yield Fraction(j,i)

def hue_to_hsvs(h: Fraction):
    # tweak values to adjust scheme
    for s in [Fraction(6,10)]:
        for v in [Fraction(6,10), Fraction(9,10)]: 
            yield (h, s, v) 

def rgb_to_css(rgb) -> str:
    uint8tuple = map(lambda y: int(y*255), rgb)
    return tuple(uint8tuple)#"({},{},{})".format(*uint8tuple)

def css_to_html(css):
    return f"<text style=background-color:{css}>&nbsp;&nbsp;&nbsp;&nbsp;</text>"

def n_colors(n=33):
    hues = infinite_hues()
    hsvs = itertools.chain.from_iterable(hue_to_hsvs(hue) for hue in hues)
    rgbs = (colorsys.hsv_to_rgb(*hsv) for hsv in hsvs)
    csss = (rgb_to_css(rgb) for rgb in rgbs)
    to_ret = list(itertools.islice(csss, n))
    return to_ret #[(float(c) for c in it) for it in to_ret]

colors = n_colors(6)
print(colors)

[(153, 61, 61), (229, 91, 91), (61, 153, 153), (91, 229, 229), (107, 153, 61), (160, 229, 91)]


In [330]:
def iris_region_by_t(t, regions):
    for r in regions:
        if r.PointInSet(t):
            return r
    return None

    
def plane_callback(t):
    cur_r = iris_region_by_t(t, certified_regions_polys.keys())
    if cur_r is not None and cur_r in certified_regions_polys.keys():
        planes = certified_regions_polys[cur_r]
        colors = n_colors(len(planes.keys()))
        for i, (geomA, geomB) in enumerate(planes.keys()):
#             plot_plane_geom_id(geomA, geomB, planes, t, color = colors[i])
            if geomA.get_value() == 132 and geomB.get_value() == 139:
                plot_plane_geom_id(geomA, geomB, planes, t, color = colors[i])
            if geomA.get_value() == 125 and geomB.get_value() == 139:
                plot_plane_geom_id(geomA, geomB, planes, t, color = colors[i])
    else:
        vis["plane"].delete()
        
def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    t = convert_q_to_t(np.array(q).reshape(1,-1)).squeeze()
    plane_callback(t)
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(t))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

In [331]:
# Solve path planning
start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start.squeeze(), target.squeeze(), regions_shrunk)
spp.set_max_velocity([.3, .3, .3])
spp.set_extra_control_points_per_region(3)

# print(spp.num_regions())
traj_iris = spp.SolveVerbose()
print(time.time() - start_time)
print(traj_iris.start_time())
print(traj_iris.end_time())
draw_traj_tspace(traj_iris, 100, 'iris')
#animate Iris + spp sol
substeps = 1000
its = 1
utils.animate_t(traj_iris, showres, substeps, convert_t_to_q, its*substeps)

0.5473332405090332
0.0
15.469461185778115


In [333]:
its = 5
utils.animate_t(traj_iris, showres, substeps, convert_t_to_q, its*substeps)

In [321]:
print(pairs)

{(<GeometryId value=132>, <GeometryId value=139>), (<GeometryId value=132>, <GeometryId value=184>), (<GeometryId value=125>, <GeometryId value=177>), (<GeometryId value=139>, <GeometryId value=184>), (<GeometryId value=132>, <GeometryId value=177>), (<GeometryId value=125>, <GeometryId value=170>), (<GeometryId value=139>, <GeometryId value=177>), (<GeometryId value=132>, <GeometryId value=170>), (<GeometryId value=125>, <GeometryId value=163>), (<GeometryId value=139>, <GeometryId value=170>), (<GeometryId value=125>, <GeometryId value=139>), (<GeometryId value=132>, <GeometryId value=163>), (<GeometryId value=139>, <GeometryId value=163>), (<GeometryId value=125>, <GeometryId value=184>)}
